# AES MixColumns as Field Operations

**Module 03** | 03-galois-fields-aes

*Column polynomial multiplication*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **aes mixcolumns as field operations**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [The AES S-Box](03d-aes-sbox-construction.ipynb).
- Concepts and notation introduced in the previous notebook.

## The MixColumns Matrix

In [ ]:
# MixColumns multiplies each column by a fixed matrix in GF(2^8)
# Matrix entries: 02, 03, 01, 01 (circulant)
# TODO: define the MixColumns matrix over GF(2^8)

## Polynomial Representation

In [ ]:
# Each column is a polynomial in GF(2^8)[x] / (x^4 + 1)
# MixColumns = multiply by c(x) = 03*x^3 + 01*x^2 + 01*x + 02
# TODO: implement polynomial multiplication mod x^4 + 1

## Step-by-Step Example

In [ ]:
# Apply MixColumns to a sample column
# TODO: walk through one column transformation

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **aes mixcolumns as field operations**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [A Full AES Round](03f-full-aes-round.ipynb)